Detailed ad retrieval from 3 websites ingatlanok.hu,ingatlan.com, otthoncentrum.

Step 1: retrive unique ad's urls from the search engine result to 'urls' variable for a give district
Step 2: use the urls to retrive the ad homepage <body> tag
Step 3: save the ads into source specific .csv

In [3]:
    #urls
    url1a = 'https://ingatlan.com/szukites/elado+lakas+xi-ker+48-55-m2+ar-szerint?page='
    url1b = 'https://ingatlan.com/szukites/elado+lakas+vi-ker+38-55-m2+ar-szerint?page='
    url2 = 'https://ingatlanok.hu/elado/lakas+haz/budapest-v-ker:25nm-tol;33nm-ig?record=0&num=100'
    url2a = 'https://ingatlanok.hu/elado/lakas+haz/budapest-xi-ker:48nm-tol;55nm-ig?record=0&num=100'
    url2b = 'https://ingatlanok.hu/elado/lakas+haz/budapest-vi-ker:38nm-tol;55nm-ig?record=0&num=100'
    url3 = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest05-kerulet/hely-id:budapest05-kerulet,/netto-alapterulet:28~33/emelet:1,8'
    url3a = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest11-kerulet/hely-id:budapest11-kerulet,/netto-alapterulet:45~55/emelet:1,8'
    url3b = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest06-kerulet/hely-id:budapest06-kerulet,/netto-alapterulet:38~55/emelet:1,8'

    url1 = 'https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page='
    url2 = 'https://ingatlanok.hu/elado/lakas+haz/budapest-v-ker:25nm-tol;33nm-ig?record=0&num=100'
    url3 = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest05-kerulet/hely-id:budapest05-kerulet,/netto-alapterulet:28~33/emelet:1,8'
    


In [9]:
import requests
#from urllib.robotparser import RobotFileParser
from bs4 import BeautifulSoup
import urllib3
import warnings
warnings.filterwarnings('ignore')
import datetime as dtime
import time
import pandas as pd
import certifi

In [2]:
#searchpage retrival

In [10]:
#ingatlan.com, multi-page
def _multi_get(url, n):
    '''
    gets multipages session based webpages like search results specialized to ingatlan.com site
    
    url for URL
    n for number of pages to follow
    
    returns ads specific url in list
    '''
    urls = []
    link = url + str(n)
    print(link)
    http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', \
                             ca_certs=certifi.where(), \
                            headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:62.0) Gecko/20100101 Firefox/62.0'})
    response = http.request('GET', link)
    soup = BeautifulSoup(response.data, 'lxml')
    l = soup.find_all('div', attrs = {'class': 'listing js-listing listing--cluster-parent js-cluster-parent'})
    l = l + (soup.find_all('div', attrs = {'class': 'listing js-listing'}))
    for i in l:
            urls.append('https://ingatlan.com'+i.find('a')['href'])
    
    return urls

def _multipagesSearch(url , v=10):
    '''
    wrapper for _multi_get(url, n) to introduce timer between page download 
    url for URL,
    v for number of pages to return
    
    _multi_get:
    gets multipages session based webpages like search results specialized to ingatlan.com site
    
    returns ad specific url in list
    '''
    i=1
    while i <= v:
        text = _multi_get(url, n=i)
        
        if 'csvtext' in locals():
            csvtext = csvtext + text
        else: csvtext = text
            
        time.sleep(1)
        i += 1
    return csvtext

#ingatlanok.hu, oc.hu, one page
def _simpleSearch(url):
    '''
    gets simple one page website from the URL
    
    returns ad specific url for .hu and oc.hu pages
    '''
    r=requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:62.0) Gecko/20100101 Firefox/62.0'})
    
    page = BeautifulSoup(r.content, 'lxml')
    body = page.body
    url_list = []
    if 'ingatlanok.hu' in url:
        a = body.find_all('div', attrs={'class': 'result-row'})
        for i in a:
            url_list.append(i.find_all('a')[1]['href'])
    if 'oc.hu' in url:
        a = body.find_all('div', attrs={'class': 'estate-list-box'})
        for i in a:
            url_list.append('https://www.oc.hu'+i.find_all('a')[0]['href'])
    return url_list

In [4]:
#compose for searPage url retrieval
def searchPage(district):
    '''
    retrives the search page results ad's url into a [list] according to the given district.
    Options:
    District: 'V', 'VI','XI'
    '''
    if district == 'V':
        url1 = 'https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page='
        url2 = 'https://ingatlanok.hu/elado/lakas+haz/budapest-v-ker:25nm-tol;33nm-ig?record=0&num=100'
        url3 = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest05-kerulet/hely-id:budapest05-kerulet,/netto-alapterulet:28~33/emelet:1,8'
    elif district == 'VI':
        url1 = 'https://ingatlan.com/szukites/elado+lakas+vi-ker+38-55-m2+ar-szerint?page='
        url2 = 'https://ingatlanok.hu/elado/lakas+haz/budapest-vi-ker:38nm-tol;55nm-ig?record=0&num=100'
        url3 = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest06-kerulet/hely-id:budapest06-kerulet,/netto-alapterulet:38~55/emelet:1,8'
    elif district == 'XI':
        url1 = 'https://ingatlan.com/szukites/elado+lakas+xi-ker+48-55-m2+ar-szerint?page='
        url2 = 'https://ingatlanok.hu/elado/lakas+haz/budapest-xi-ker:48nm-tol;55nm-ig?record=0&num=100'
        url3 = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest11-kerulet/hely-id:budapest11-kerulet,/netto-alapterulet:45~55/emelet:1,8'
    
    hu = _simpleSearch(url2)
    print('Hu search: ',len(hu))
    
    com = _multipagesSearch(url1)
    print('Com search: ',len(com))
    
    cen = _simpleSearch(url3)
    print('Cen search: ',len(cen))
    
    alls = [*hu,*com,*cen]
    print('All urls: ', len(alls))
    
    alls =[i for i in alls if 'http' in i]
    return alls

In [5]:
# detailed webpage retrival

In [6]:
def _simple(url):
    '''
    Returns the page <body> tag from url retrieval
    '''
    page = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:62.0) Gecko/20100101 Firefox/62.0'})
    body = BeautifulSoup(page.content, 'lxml').body

    return body

def details(urls, district):
    '''
    Go through the list of urls, retrieves the body tag. 
    Sorted by url, saves the body tag of those that has the keyword specific to the url source 
    
    district: 'V', 'VI', 'XI'
    '''
    
    adshu = []
    adscom = []
    adscen = []
    n=0
    for i in urls:
        print(n,'\t',i)
        if 'ingatlanok.hu' in i:
            try:
                back = _simple(i)               
                txt = back.get_text()

                if 'Azonosító:' in txt:             
                    adshu.append(back)
                    
            except: pass
            
        elif 'oc.hu' in i:
            try:
                back = _simple(i)               
                txt = back.get_text()
                
                if 'Regisztrációs szám:' in txt:             
                    adscen.append(back)
            except: pass
            
        else:
            try:
                back = _simple(i)               
                txt = back.get_text()
                
                if 'Parkolás' in txt or 'Ingatlan állapota' in txt:             
                    adscom.append(back)
            except: pass
        
        n+=1 # counter
    
    now = dtime.datetime.now()
    m = [str(now.month) if len(str(now.month)) == 2 else '0' + str(now.month)]
    d = [str(now.day) if len(str(now.day)) == 2 else '0' + str(now.day)]
    y = str(now.year)
    
    with open('adshu'+ district + '_' + d[0] + m[0] + y + '.csv', 'w') as f:
                f.write('%s' % adshu)
    with open('adscom'+ district + '_' + d[0] + m[0] + y + '.csv', 'w') as f:
                f.write('%s' % adscom)
    with open('adscen'+ district + '_' + d[0] + m[0] + y + '.csv', 'w') as f:
                f.write('%s' % adscen)
    return adshu, adscom,adscen

In [7]:
def checkDownload(date, d = ['V','VI','XI']):
    '''
    Check the No of saved ads.
    Search by date
    Search by district in list format -> ['XX']
    '''
    for n in d:
        for i in ['adshu'+n,'adscom'+n,'adscen'+n]:
            try:
                f = i+ '_' + date +'.csv'
                with open(f, 'r') as r:
                    l = r.read()
                ll = l.split('</body>,')
                print('Saved ads in {}: {}'.format(i,len(ll)))
            except: print('No such file ' + f)

In [11]:
ads = []
for i in ['V','VI', 'XI']:
    urls = searchPage(i)
    ads.append(details(urls,i))

Hu search:  16
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=1
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=2
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=3
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=4
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=5
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=6
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=7
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=8
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=9
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=10
Com search:  50
Cen search:  4
All urls:  70
0 	 https://ingatlanok.hu/elado/lakas/budapest_v_ker/64-millio-ft;31-negyzetmeter;1-szoba;tegla-epitesu/10019487
1 	 https://ingatlanok.hu/elado/lakas/budapest_v_ker/27-millio-ft;26-negyzet

In [12]:
import os
for i in sorted(os.listdir()):
    if 'ads' in i:
        print(i,round(os.stat(i).st_size/(1024*1024),2), 'M')

adscenVI_03032020.csv 5.18 M
adscenVI_03052020.csv 5.26 M
adscenVI_05042020.csv 5.25 M
adscenVI_08032020.csv 5.19 M
adscenVI_09062020.csv 5.31 M
adscenVI_10052020.csv 5.28 M
adscenVI_12042020.csv 5.25 M
adscenVI_15032020.csv 5.23 M
adscenVI_17052020.csv 5.29 M
adscenVI_19042020.csv 5.24 M
adscenVI_22032020.csv 5.24 M
adscenVI_24052020.csv 5.29 M
adscenVI_26042020.csv 5.26 M
adscenVI_29032020.csv 5.25 M
adscenVI_31052020.csv 5.32 M
adscenV_01032020.csv 0.2 M
adscenV_03052020.csv 0.42 M
adscenV_05042020.csv 0.3 M
adscenV_08032020.csv 0.2 M
adscenV_09062020.csv 0.42 M
adscenV_10052020.csv 0.42 M
adscenV_12042020.csv 0.3 M
adscenV_15032020.csv 0.2 M
adscenV_17052020.csv 0.54 M
adscenV_19042020.csv 0.3 M
adscenV_24022020.csv 0.18 M
adscenV_24052020.csv 0.54 M
adscenV_26042020.csv 0.42 M
adscenV_29032020.csv 0.3 M
adscenV_31052020.csv 0.42 M
adscenXI_03052020.csv 2.79 M
adscenXI_04032020.csv 2.96 M
adscenXI_05042020.csv 2.78 M
adscenXI_08032020.csv 3.06 M
adscenXI_09062020.csv 3.12 M
adscenX

In [14]:
checkDownload('30032020', ['XI'])

Saved ads in adshuXI: 100
Saved ads in adscomXI: 200
Saved ads in adscenXI: 25


In [14]:
ads

[([], [], []), ([], [], []), ([], [], [])]